# exp35

下記のStatedate特徴量を作成
https://www.kaggle.com/code/thedevastator/amex-features-the-best-of-both-worlds



In [1]:
# ====================================================
# Library
# ====================================================
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools


In [2]:

# ====================================================
# Get the difference
# ====================================================
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data():
    train = pd.read_parquet('../input/AMEXdata-integerdtypes-parquetformat/train.parquet')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    # Get the difference
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_diff
    gc.collect()
    
    test = pd.read_parquet('../input/AMEXdata-integerdtypes-parquetformat/test.parquet')
    print('Starting test feature engineer...')
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
    # Get the difference
    test_diff = get_difference(test, num_features)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()
    # Save files to disk
    train.to_parquet('../input/best of both worlds/train_fe.parquet')
    test.to_parquet('..Y/input/best of both worlds/test_fe.parquet')

# Read & Preprocess Data
# read_preprocess_data()

In [3]:
train = pd.read_parquet("../input/best of both worlds/train_fe.parquet")
exp35feature = train.columns.to_list()
train.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,...,D_138_std,D_138_min,D_138_max,D_138_last,D_139_mean,D_139_std,D_139_min,D_139_max,D_139_last,D_140_mean,D_140_std,D_140_min,D_140_max,D_140_last,D_141_mean,D_141_std,D_141_min,D_141_max,D_141_last,D_142_mean,D_142_std,D_142_min,D_142_max,D_142_last,D_143_mean,D_143_std,D_143_min,D_143_max,D_143_last,D_144_mean,D_144_std,D_144_min,D_144_max,D_144_last,D_145_mean,D_145_std,D_145_min,D_145_max,D_145_last,B_30_count,B_30_last,B_30_nunique,B_38_count,B_38_last,B_38_nunique,D_114_count,D_114_last,D_114_nunique,D_116_count,D_116_last,D_116_nunique,D_117_count,D_117_last,D_117_nunique,D_120_count,D_120_last,D_120_nunique,D_126_count,D_126_last,D_126_nunique,D_63_count,D_63_last,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_diff1,B_17_diff1,B_18_diff1,B_19_diff1,B_20_diff1,S_12_diff1,R_6_diff1,S_13_diff1,B_21_diff1,D_69_diff1,B_22_diff1,D_70_diff1,D_71_diff1,D_72_diff1,S_15_diff1,B_23_diff1,D_73_diff1,P_4_diff1,D_74_diff1,D_75_diff1,D_76_diff1,B_24_diff1,R_7_diff1,D_77_diff1,B_25_diff1,B_26_diff1,D_78_diff1,D_79_diff1,R_8_diff1,R_9_diff1,S_16_diff1,D_80_diff1,R_10_diff1,R_11_diff1,B_27_diff1,D_81_diff1,D_82_diff1,S_17_di

In [9]:
aa = pd.read_parquet( '../input/amex-fe/train_fe.parquet')
exp03feature = aa.columns.to_list()
# aa.head

In [15]:
# for i in exp03feature:


num_features1_test = [col for col in exp35feature if col not in exp03feature]
# num_features1_test = [col for col in exp35feature if col in "diff"]


# diff系の特徴量が抽出される
len(num_features1_test)

177

In [16]:
# aa[num_features1_test]
# num_features1_testnum_features1_test
num_features1_test.append("customer_ID")

train_diff = train[num_features1_test]
train_diff.to_parquet('../input/best of both worlds/train_diff.parquet')

In [18]:
test = pd.read_parquet("../input/best of both worlds/test_fe.parquet")
# exp35feature = test.columns.to_list()
test.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,...,D_138_mean,D_138_std,D_138_min,D_138_max,D_138_last,D_139_mean,D_139_std,D_139_min,D_139_max,D_139_last,D_140_mean,D_140_std,D_140_min,D_140_max,D_140_last,D_141_mean,D_141_std,D_141_min,D_141_max,D_141_last,D_142_mean,D_142_std,D_142_min,D_142_max,D_142_last,D_143_mean,D_143_std,D_143_min,D_143_max,D_143_last,D_144_mean,D_144_std,D_144_min,D_144_max,D_144_last,D_145_mean,D_145_std,D_145_min,D_145_max,D_145_last,B_30_count,B_30_last,B_30_nunique,B_38_count,B_38_last,B_38_nunique,D_114_count,D_114_last,D_114_nunique,D_116_count,D_116_last,D_116_nunique,D_117_count,D_117_last,D_117_nunique,D_120_count,D_120_last,D_120_nunique,D_126_count,D_126_last,D_126_nunique,D_63_count,D_63_last,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_diff1,B_17_diff1,B_18_diff1,B_19_diff1,B_20_diff1,S_12_diff1,R_6_diff1,S_13_diff1,B_21_diff1,D_69_diff1,B_22_diff1,D_70_diff1,D_71_diff1,D_72_diff1,S_15_diff1,B_23_diff1,D_73_diff1,P_4_diff1,D_74_diff1,D_75_diff1,D_76_diff1,B_24_diff1,R_7_diff1,D_77_diff1,B_25_diff1,B_26_diff1,D_78_diff1,D_79_diff1,R_8_diff1,R_9_diff1,S_16_diff1,D_80_diff1,R_10_diff1,R_11_diff1,B_27_diff1,D_81_diff1,D_82_di

In [19]:

test_diff = test[num_features1_test]
test_diff.head()
# 

,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_diff1,B_17_diff1,B_18_diff1,B_19_diff1,B_20_diff1,S_12_diff1,R_6_diff1,S_13_diff1,B_21_diff1,D_69_diff1,B_22_diff1,D_70_diff1,D_71_diff1,D_72_diff1,S_15_diff1,B_23_diff1,D_73_diff1,P_4_diff1,D_74_diff1,D_75_diff1,D_76_diff1,B_24_diff1,R_7_diff1,D_77_diff1,B_25_diff1,B_26_diff1,D_78_diff1,D_79_diff1,R_8_diff1,R_9_diff1,S_16_diff1,D_80_diff1,R_10_diff1,R_11_diff1,B_27_diff1,D_81_diff1,D_82_diff1,S_17_diff1,R_12_diff1,B_28_diff1,R_13_diff1,D_83_diff1,R_14_diff1,R_15_diff1,D_84_diff1,R_16_diff1,B_29_diff1,S_18_diff1,D_86_diff1,D_87_diff1,R_17_diff1,R_18_diff1,D_88_diff1,B_31_diff1,S_19_diff1,R_19_diff1,B_32_diff1,S_20_diff1,R_20_diff1,R_21_diff1,B_33_diff1,D_89_diff1,R_22_diff1,R_23_diff1,D_91_diff1,D_92_diff1,D_93_diff1,D_94_diff1,R_24_diff1,R_25_diff1,D_96_diff1,S_22_diff1,S_23_diff1,S_24_diff1,S_25_diff1,S_26_diff1,D_102_diff1,D_103_diff1,D_104_diff1,D_105_diff1,D_106_diff1,D_107_diff1,B_36_diff1,B_37_diff1,R_26_diff1,R_27_diff1,D_108_diff1,D_109_diff1,D_110_diff1,D_111_diff1,B_39_diff1,D_112_diff1,B_40_diff1,S_27_diff1,D_113_diff1,D_115_diff1,D_118_diff1,D_119_diff1,D_121_diff1,D_122_diff1,D_123_diff1,D_124_diff1,D_125_diff1,D_127_diff1,D_128_diff1,D_129_diff1,B_41_diff1,B_42_diff1,D_130_diff1,D_131_diff1,D_132_diff1,D_133_diff1,R_28_diff1,D_134_diff1,D_135_diff1,D_136_diff1,D_137_diff1,D_138_diff1,D_139_diff1,D_140_diff1,D_141_diff1,D_142_diff1,D_143_diff1,D_144_diff1,D_145_diff1,customer_ID
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.028144,4.0,-0.006647,0.008677,0.000000,-0.008592,0.0,-0.008386,-0.013312,0.001126,0.0,-1.0,0.008003,0.044772,0.0,0.004869,0.007213,-0.002172,0.0,0.001855,0.006839,0.000546,NaN,1.0,0.003786,0.0,0.001534,-0.073692,-0.003809,NaN,0.002261,0.000623,0.0,0.0,0.0,-0.008975,-0.008802,458.0,0.023351,NaN,-0.004477,0.0,0.004878,0.003691,-0.011465,0.0,-0.027946,0.011317,-0.006861,1.0,0.013832,0.0,0.0,NaN,-0.000258,0.0,0.0,-0.002092,-0.001788,0.0,0.006563,0.001590,0.0,0.0,-0.009105,0.0,-1.0,0.007484,-0.007067,0.00552,0.0,0.0,NaN,0.000946,0.0,NaN,0.004265,0.000406,0.0,0.0,0.0,0.0,0.005181,0.0,0.0,0.0,-0.002625,0.0,0.0,0.002523,0.0,-0.006575,0.0,0.0,0.0,0.0,0.0,0.0,-0.005911,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-0.001986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.007506,0.001997,-0.007330,-0.007122,-0.003119,0.002910,0.0,0.000000,NaN,0.0,0.0,-0.007995,-0.003831,0.0,-0.005000,0.0,0.0,NaN,0.0,NaN,0.000000,-0.006724,-0.023175,0.0,-0.007683,-0.007760,0.004945,0.000492,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,0.000000,0.0,NaN,0.002647,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,0.0,-0.002044,0.0,00000469ba478561f23a92a868bd366de6f6527a684c9a...
1,-0.019865,-3.0,-0.037400,0.008463,0.000000,-0.001867,0.0,0.005265,NaN,NaN,0.0,-4.0,0.002990,0.108394,0.0,0.031272,0.004706,0.037209,0.0,-0.016004,-0.021462,0.000701,NaN,0.0,-0.112125,0.0,-0.007925,-0.096179,-0.004594,NaN,0.000668,-0.038763,0.0,0.0,0.0,0.001613,0.002045,-292.0,0.000313,-0.002864,-0.000583,0.0,-0.000596,-0.005013,-0.039476,-2.0,0.020737,0.046098,0.002661,-4.0,0.007243,0.0,1.0,NaN,0.000000,0.0,0.0,-0.000126,0.008638,-145.0,-0.001356,-0.000650,0.0,0.0,0.001428,0.0,0.0,-0.023045,NaN,0.00000,0.0,0.0,NaN,0.001512,0.0,-0.003998,-0.059654,-0.001261,0.0,0.0,0.0,0.0,-0.002832,0.0,0.0,0.0,-0.009089,0.0,0.0,0.000365,0.0,-0.032622,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.

In [20]:
test_diff.to_parquet('../input/best of both worlds/test_diff.parquet')

# Training & Inference

In [3]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

import pickle

# ====================================================
# Configurations
# ====================================================
class CFG:
    
    
    input_dir = '../feature/exp03_amex-fe/'
    output_dir = '../output/exp33_statedate_lgb/'
    seed = 42
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'
    model = "lgb"
    ver = "exp33"

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe_plus_plus.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe_plus_plus.parquet')
    return train, test

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True


In [4]:
# seed_everything(CFG.seed)
# train, test = read_data()


# date_train = pd.read_pickle('../feature/exp18_3_tsfresh/train_variation_coefficient.pkl')
# date_test = pd.read_pickle('../feature/exp18_3_tsfresh/test_variation_coefficient.pkl')

# train = train.merge(date_train,on = "customer_ID",how = "left")
# test = test.merge(date_test,on = "customer_ID",how = "left")

# del date_train,date_test
# gc.collect

train = pd.read_pickle('../feature/exp30_statementdate/statedate_train.pkl')
test0 = pd.read_pickle('../feature/exp30_statementdate/statedate_test0.pkl')
test1 = pd.read_pickle('../feature/exp30_statementdate/statedate_test1.pkl')
test2 = pd.read_pickle('../feature/exp30_statementdate/statedate_test2.pkl')
test3 = pd.read_pickle('../feature/exp30_statementdate/statedate_test3.pkl')

test = pd.concat([test0,test1,test2,test3],axis = 0)


del test0,test1,test2,test3
gc.collect()

5259

In [5]:
test.head()

,S_2,LT,B_11_last,B_11_first,S_12_last,S_12_first,D_46_last,D_46_first,S_27_last,S_27_first,D_53_last,D_53_first,S_20_last,S_20_first,B_39_last,B_39_first,R_19_last,R_19_first,B_13_last,B_13_first,D_45_last,D_45_first,B_12_last,B_12_first,B_28_last,B_28_first,S_16_last,S_16_first,B_15_last,B_15_first,B_3_last,B_3_first,R_26_last,R_26_first,D_140_last,D_140_first,R_2_last,R_2_first,D_125_last,D_125_first,D_127_last,D_127_first,R_11_last,R_11_first,R_12_last,R_12_first,R_4_last,R_4_first,D_132_last,D_132_first,R_8_last,R_8_first,B_1_last,B_1_first,S_6_last,S_6_first,D_47_last,D_47_first,S_8_last,S_8_first,B_32_last,B_32_first,D_83_last,D_83_first,D_71_last,D_71_first,R_9_last,R_9_first,B_22_last,B_22_first,D_75_last,D_75_first,B_16_last,B_16_first,D_133_last,D_133_first,D_73_last,D_73_first,R_3_last,R_3_first,D_96_last,D_96_first,D_119_last,D_119_first,D_122_last,D_122_first,S_26_last,S_26_first,D_106_last,D_106_first,R_6_last,R_6_first,D_138_last,D_138_first,S_11_last,S_11_first,D_72_last,D_72_first,B_14_last,B_14_first,B_33_last,B_33_first,B_2_last,B_2_first,D_76_last,D_76_first,B_37_last,B_37_first,D_81_last,D_81_first,B_20_last,B_20_first,D_70_last,D_70_first,D_121_last,D_121_first,D_86_last,D_86_first,B_21_last,B_21_first,B_5_last,B_5_first,B_19_last,B_19_first,D_65_last,D_65_first,R_7_last,R_7_first,B_17_last,B_17_first,D_124_last,D_124_first,P_4_last,P_4_first,P_3_last,P_3_first,B_42_last,B_42_first,S_13_last,S_13_first,S_22_last,S_22_first,D_59_last,D_59_first,B_41_last,B_41_first,S_24_last,S_24_first,S_5_last,S_5_first,D_49_last,D_49_first,D_78_last,D_78_first,B_26_last,B_26_first,D_142_last,D_142_first,D_69_last,D_69_first,D_54_last,D_54_first,R_10_last,R_10_first,D_56_last,D_56_first,B_24_last,B_24_first,S_19_last,S_19_first,B_8_last,B_8_first,S_7_last,S_7_first,D_50_last,D_50_first,D_112_last,D_112_first,D_79_last,D_79_first,D_42_last,D_42_first,R_1_last,R_1_first,D_60_last,D_60_first,D_134_last,D_134_first,P_2_last,P_2_first,D_131_last,D_131_first,B_23_last,B_23_first,D_80_last,D_80_first,D_82_last,D_82_first,D_41_last,D_41_first,B_6_last,B_6_first,D_77_last,D_77_first,D_105_last,D_105_first,D_91_last,D_91_first,D_58_last,D_58_first,B_9_last,B_9_first,D_55_last,D_55_first,R_15_last,R_15_first,D_48_last,D_48_first,B_4_last,B_4_first,D_39_last,D_39_first,D_62_last,D_62_first,D_43_last,D_43_first,R_13_last,R_13_first,R_14_last,R_14_first,S_3_last,S_3_first,B_10_last,B_10_first,R_20_last,R_20_first,R_5_last,R_5_first,D_44_last,D_44_first,D_52_last,D_52_first,R_27_last,R_27_first,B_18_last,B_18_first,D_61_last,D_61_first,B_36_last,B_36_first,...,B_26_last_first_div,D_142_last_first_div,D_142_last_mean_sub,D_142_last_mean_div,D_142_last_max_div,D_69_last_first_div,D_69_last_mean_sub,D_69_last_mean_div,D_54_last_first_div,D_54_last_mean_sub,D_54_last_mean_div,R_10_last_first_div,R_10_last_mean_sub,R_10_last_mean_div,R_10_last_max_div,D_56_last_first_div,B_24_last_first_div,B_24_last_mean_sub,B_24_last_mean_div,B_24_last_max_div,S_19_last_first_div,B_8_last_first_div,B_8_last_mean_sub,B_8_last_mean_div,B_8_last_max_div,S_7_last_first_div,S_7_last_mean_sub,S_7_last_mean_div,S_7_last_max_div,S_7_last_min_div,B_11-P_2_last_first_div,B_11-P_2_last_mean_sub,B_11-P_2_last_mean_div,B_11-P_2_last_max_div,B_11-P_2_last_min_div,D_50_last_first_div,D_50_last_mean_sub,D_50_last_mean_div,D_50_last_max_div,D_50_last_min_div,D_112_last_first_div,D_112_last_mean_sub,D_112_last_mean_div,D_112_last_max_div,D_112_last_min_div,D_79_last_first_div,D_42_last_first_div,D_42_last_mean_sub,D_42_last_mean_div,D_42_last_max_div,D_42_last_min_div,R_1_last_first_div,R_1_last_mean_sub,R_1_last_mean_div,R_1_last_max_div,R_1_last_min_div,D_60_last_first_div,D_60_last_mean_sub,D_60_last_mean_div,D_60_last_max_div,D_60_last_min_div,D_134_last_first_div,D_134_last_mean_sub,D_134_last_mean_div,D_134_last_max_div,P_2_last_first_div,P_2_last_mean_sub,P_2_last_mean_div,P_2_last_max_div,P_2_last_min_div,D_131_last_first_div,D_131_last_mean_sub,D_131_last_mean_div

In [5]:

# date_train = pd.read_pickle('../feature/Statement Dates/train_SDist.pkl')
# date_test = pd.read_pickle('../feature/Statement Dates/test_SDist.pkl')

# train = train.merge(date_train,on = "customer_ID",how = "left")
# test = test.merge(date_test,on = "customer_ID",how = "left")

In [6]:
train.head()

,customer_ID,S_2,LT,B_33_last,B_33_first,D_41_last,D_41_first,D_79_last,D_79_first,R_12_last,R_12_first,B_13_last,B_13_first,R_1_last,R_1_first,S_8_last,S_8_first,D_59_last,D_59_first,S_20_last,S_20_first,D_54_last,D_54_first,B_22_last,B_22_first,D_50_last,D_50_first,D_80_last,D_80_first,B_14_last,B_14_first,R_11_last,R_11_first,D_130_last,D_130_first,B_11_last,B_11_first,B_8_last,B_8_first,D_78_last,D_78_first,S_11_last,S_11_first,B_18_last,B_18_first,B_17_last,B_17_first,B_39_last,B_39_first,B_6_last,B_6_first,R_14_last,R_14_first,S_23_last,S_23_first,D_127_last,D_127_first,D_61_last,D_61_first,S_16_last,S_16_first,D_75_last,D_75_first,D_86_last,D_86_first,D_105_last,D_105_first,D_73_last,D_73_first,D_82_last,D_82_first,D_58_last,D_58_first,P_3_last,P_3_first,D_39_last,D_39_first,D_112_last,D_112_first,D_62_last,D_62_first,S_12_last,S_12_first,D_53_last,D_53_first,B_2_last,B_2_first,D_42_last,D_42_first,D_124_last,D_124_first,R_26_last,R_26_first,B_21_last,B_21_first,R_8_last,R_8_first,D_125_last,D_125_first,R_27_last,R_27_first,D_133_last,D_133_first,D_52_last,D_52_first,S_5_last,S_5_first,R_15_last,R_15_first,D_141_last,D_141_first,B_7_last,B_7_first,S_24_last,S_24_first,D_83_last,D_83_first,S_26_last,S_26_first,B_36_last,B_36_first,R_7_last,R_7_first,B_41_last,B_41_first,S_27_last,S_27_first,D_70_last,D_70_first,D_48_last,D_48_first,D_55_last,D_55_first,D_56_last,D_56_first,D_71_last,D_71_first,D_134_last,D_134_first,B_37_last,B_37_first,D_131_last,D_131_first,R_9_last,R_9_first,B_16_last,B_16_first,B_25_last,B_25_first,D_72_last,D_72_first,D_47_last,D_47_first,B_24_last,B_24_first,D_96_last,D_96_first,D_119_last,D_119_first,B_40_last,B_40_first,D_142_last,D_142_first,P_2_last,P_2_first,D_43_last,D_43_first,D_106_last,D_106_first,S_13_last,S_13_first,B_32_last,B_32_first,B_1_last,B_1_first,B_23_last,B_23_first,S_6_last,S_6_first,D_140_last,D_140_first,B_9_last,B_9_first,D_138_last,D_138_first,S_22_last,S_22_first,R_4_last,R_4_first,R_20_last,R_20_first,B_26_last,B_26_first,D_46_last,D_46_first,D_60_last,D_60_first,P_4_last,P_4_first,B_20_last,B_20_first,B_19_last,B_19_first,D_45_last,D_45_first,D_91_last,D_91_first,D_132_last,D_132_first,S_7_last,S_7_first,R_6_last,R_6_first,D_44_last,D_44_first,B_10_last,B_10_first,R_13_last,R_13_first,B_42_last,B_42_first,B_12_last,B_12_first,S_19_last,S_19_first,R_19_last,R_19_first,D_145_last,D_145_first,R_10_last,R_10_first,D_122_last,D_122_first,B_28_last,B_28_first,B_4_last,B_4_first,R_5_last,R_5_first,D_49_last,D_49_first,D_76_last,D_76_first,D_69_last,D_69_first,D_121_last,D_121_first,B_5_last,B_5_first,R_3_last,...,B_24_last_mean_sub,B_24_last_mean_div,B_24_last_max_div,D_96_last_first_div,D_96_last_mean_sub,D_96_last_mean_div,D_119_last_first_div,D_119_last_mean_sub,D_119_last_mean_div,D_119_last_max_div,D_119_last_min_div,B_40_last_first_div,B_40_last_mean_sub,B_40_last_mean_div,B_40_last_max_div,D_142_last_first_div,D_142_last_mean_sub,D_142_last_mean_div,D_142_last_max_div,P_2_last_first_div,P_2_last_mean_sub,P_2_last_mean_div,P_2_last_max_div,P_2_last_min_div,D_43_last_first_div,D_43_last_mean_sub,D_43_last_mean_div,D_43_last_max_div,D_106_last_first_div,S_13_last_first_div,S_13_last_mean_sub,S_13_last_mean_div,S_13_last_max_div,B_32_last_first_div,B_32_last_mean_sub,B_32_last_mean_div,B_1_last_first_div,B_1_last_mean_sub,B_1_last_mean_div,B_1_last_max_div,B_23_last_first_div,B_23_last_mean_sub,B_23_last_mean_div,B_23_last_max_div,S_16-P_3_last_first_div,S_16-P_3_last_mean_sub,S_16-P_3_last_mean_div,S_16-P_3_last_max_div,S_16-P_3_last_min_div,S_6_last_first_div,S_6_last_mean_sub,S_6_last_mean_div,B_17-P_3_last_first_div,B_17-P_3_last_mean_sub,B_17-P_3_last_mean_div,B_17-P_3_last_max_div,B_17-P_3_last_min_div,D_140_last_first_div,D_140_last_mean_sub,D_140_last_mean_div,D_140_last_max_div,D_140_last_min_div,B_9_last_first_div,B_9_last_mean_sub,B_9_last_mean_div,B_9_last_max_div,B_9_last_min_div,D_138_last_first_div,S_22_last_first_div,S_22_last_mean_sub,S_22_last_mean_div,S_2

In [7]:

##LightGBMのEarlyStopping実装


class DartEarlyStopping(object):
    """DartEarlyStopping"""

    def __init__(self, data_name, monitor_metric, stopping_round):
        self.data_name = data_name
        self.monitor_metric = monitor_metric
        self.stopping_round = stopping_round
        self.best_score = None
        self.best_model = None
        self.best_score_list = []
        self.best_iter = 0

    def _is_higher_score(self, metric_score, is_higher_better):
        if self.best_score is None:
            return True
        return (self.best_score < metric_score) if is_higher_better else (self.best_score > metric_score)

    def _deepcopy(self, x):
        # copy.deepcopyではlightgbmのモデルは完全にコピーされないためpickleを使用
        return pickle.loads(pickle.dumps(x))

    def __call__(self, env):
        evals = env.evaluation_result_list
        for data, metric, score, is_higher_better in evals:
            if data != self.data_name or metric != self.monitor_metric:
                continue
            if not self._is_higher_score(score, is_higher_better):
                if env.iteration - self.best_iter > self.stopping_round:
                    # 終了させる
                    eval_result_str = '\t'.join([lgb.callback._format_eval_result(x) for x in self.best_score_list])
                    lgb.basic._log_info(f"Early stopping, best iteration is:\n[{self.best_iter+1}]\t{eval_result_str}") 
                    lgb.basic._log_info(f"You can get best model by \"DartEarlyStopping.best_model\"")
                    raise lgb.callback.EarlyStopException(self.best_iter, self.best_score_list)
                return
            # dartでは過去の木も更新されてしまうため、deepcopyしておく
            self.best_model = self._deepcopy(env.model)
            self.best_score_list = evals
            self.best_score = score
            return
        raise ValueError("monitoring metric not found")

In [ ]:



cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]

# kmeans_list = ["kmeans pred 2","kmeans pred 3","kmeans pred 4"]

cat_features = [f"{cf}_last" for cf in cat_features]
# cat_features.extend(kmeans_list)

for cat_col in cat_features:
#     print(cat_col)
    encoder = LabelEncoder()
    train[cat_col] = encoder.fit_transform(train[cat_col])
    test[cat_col] = encoder.transform(test[cat_col])


features = [col for col in train.columns if col not in ['customer_ID','S_2', CFG.target]]
params = {
    'objective': 'binary',
    'metric': CFG.metric,
    'boosting': CFG.boosting_type,
    'seed': CFG.seed,
    'num_leaves': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.20,
    'bagging_freq': 10,
    'bagging_fraction': 0.50,
    'n_jobs': -1,
    'lambda_l2': 2,
    'min_data_in_leaf': 40,
    }
# Create a numpy array to store test predictions
test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))

cids = []
tr_target = []

kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(features)} features...')
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
    lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
    lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
    
#     des = DartEarlyStopping("valid_1", CFG.metric, 1000)
    
    model = lgb.train(
        params = params,
        train_set = lgb_train,
        num_boost_round = 10500,#10500
        valid_sets = [lgb_train, lgb_valid],
#         callbacks = [des],
        early_stopping_rounds = 1500,
#         eval_metric=[lgb_amex_metric],
        verbose_eval = 500,
        feval = lgb_amex_metric
        )
    
    # Save best model
    joblib.dump(model, f'{CFG.output_dir}lgbm_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
    # Predict validation
    val_pred = model.predict(x_val)
    # Add to out of folds array
    oof_predictions[val_ind] = val_pred
    
    cids.extend(train["customer_ID"].loc[val_ind])
    tr_target.extend(train["target"].loc[val_ind])
    
    # Predict the test set
    test_pred = model.predict(test[features])
    test_predictions += test_pred / CFG.n_folds
    # Compute fold metric
    score = amex_metric(y_val, val_pred)
    print(f'Our fold {fold} CV score is {score}')
    del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
    gc.collect()
    
# Compute out of folds metric
score = amex_metric(train[CFG.target], oof_predictions)
print(f'Our out of folds CV score is {score}')


# Create a dataframe to store test prediction
test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
test_df.to_csv(f'{CFG.output_dir}test_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


dic_oof = {
    "customer_ID":cids,
    "target":tr_target,
    "tabnet_oot":oof_predictions
}

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame(dic_oof)
# oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
oof_df.to_csv(f'{CFG.output_dir}oof_{CFG.ver}_{CFG.model}_{score}_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


# # Create a dataframe to store out of folds predictions
# oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
# oof_df.to_csv(f'../output/Amex LGBM Dart CV 0.7977/oof_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


# # Create a dataframe to store test prediction
# test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'../output/Amex LGBM Dart CV 0.7977/test_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)


 
--------------------------------------------------
Training fold 0 with 1581 features...
[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.947995 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 313117
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 1580
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523
[500]	training's binary_logloss: 0.334653	training's amex_metric: 0.78057	valid_1's binary_logloss: 0.338362	valid_1's amex_metric: 0.765071
[1000]	training's binary_logloss: 0.244161	training's amex_metric: 0.796699	valid_1's binary_logloss: 0.252348	valid_1's amex_metric: 0.774877
[1500]	training's binary_logloss: 0.22035	training's amex_metric: 0.810691	valid_1's binary_logloss: 0.232759	valid_1's am

In [ ]:
x_train.head()